In [1]:
from  netCDF4 import Dataset
import  numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import joblib
import glob
import os

In [2]:
### Load in the training data from nc files. 
def nc_load(dir = "/home/jamie/cruise_processing/falkortoo/FKt240817/wh300/wh300.nc"):
    ### Load in the test data u and v for plotting. 
    file_id = Dataset(dir)
    pg = file_id.variables["pg"][:]
    amp = file_id.variables["amp"][:]
    mask = file_id.variables["pflag"][:]
    depth = file_id.variables["depth"][:]
    # u = file_id.variables["u"][:]
    # v = file_id.variables["v"][:]
    file_id.close()

    features = np.stack([amp.flatten(), depth.flatten(), pg.flatten()], axis=1) # depth.flatten()
    predictor =  mask.flatten()
    # error velocity sounds nice too
    return(features, predictor)

# funtion for listing all nc files in a directory with there absolute path. 
def list_nc_files(directory):
    nc_files = glob.glob(os.path.join(directory, '**', '*.nc'), recursive=True)
    # Convert to absolute paths
    nc_files = [os.path.abspath(path) for path in nc_files]
    return nc_files

def bulkload_nc(dir = "/home/jamie/projects/flag_model/data/ncfiles"):
    # load train data
    # loop through the list using nc_loader and put output into an empty list then compress the list. 
    dbfiles = list_nc_files(dir)
    X = list()
    Y = list()
    for i in range(0, len(dbfiles)):
        file = dbfiles[i]
        param, predic = nc_load(file)
        X.append(param)
        Y.append(predic)
    features = np.vstack(X)
    predictor = np.hstack(Y)
    predictor = predictor.flatten()
    return(features, predictor)

def what_flags(x, y, arg = [0, 1, 2, 3, 4, 5, 6, 7]):
    # Only train on the good stuf. 
    idx = np.isin(y, arg)
    usr_mask = idx.flatten()
    x = x[idx, :]
    # make all masks 0 or 1. Needed if other flags are used. 
    y = y[usr_mask]
    y[y != 0] = 1
    return(x, y)

In [3]:
# load the features
features, predictor = bulkload_nc(dir = "/home/jamie/projects/flag_model/data/ncfiles")
features, predictor = what_flags(features, predictor, arg = [0, 1])

# scale the data
# scaler = MinMaxScaler(feature_range=(0, 1))
# scaled_features = scaler.fit_transform(features)

# run the model
# X_train, X_test, y_train, y_test = train_test_split(features, predictor, test_size=0.7, random_state=42, stratify=predictor)
X_train, X_test, y_train, y_test = train_test_split(features, predictor, test_size=0.7, random_state=42, stratify=predictor)
# look into ridge and teapot. 
model = LogisticRegression(max_iter=10000, class_weight='balanced')  
model.fit(X_train, y_train)

# Save the model. 
joblib.dump(model, 'logit_model.pkl')

['logit_model.pkl']